In [1]:
# PyTorch 라이브러리 import  *** 해당 cell을 수정하지 말 것 ***
import torch
import torch.nn as nn
from torchvision import transforms, datasets

In [31]:
# TODO: CIFAR-10 training set 불러오기

batch_size=32
train_dataset = datasets.CIFAR10(root='.', train=False, download=True, transform=transforms.ToTensor())

subset = int(len(train_dataset) * 0.8)

train_loader = torch.utils.data.DataLoader(dataset=torch.utils.data.Subset(train_dataset, range(subset)), batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(dataset=torch.utils.data.Subset(train_dataset, range(subset, len(train_dataset))), batch_size=batch_size, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else ("mps" if torch.backends.mps.is_available else "cpu"))

Files already downloaded and verified


In [58]:
class MLP(nn.Module):
  def __init__(self):
    super(MLP, self).__init__()
    torch.manual_seed(2024) # 결과 재현을 위한 seed number 고정 *** 해당 line을 수정하지 말 것 ***
    # TODO: MLP 구성 layer들 선언 (Hint : Model input이 RGB 3채널 32x32 픽셀 이미지이므로 MLP 입력층 노드 개수는 32*32*3 = 3,072개임)
    self.affine1 = nn.Linear(3072, 1024)
    self.act1 = nn.ReLU()
    self.batch1 = nn.BatchNorm1d(1024)
    self.dropout1 = nn.Dropout(0.2)
    
    self.affine2 = nn.Linear(1024, 512)
    self.act2 = nn.ReLU()
    self.batch2 = nn.BatchNorm1d(512)
    self.dropout2 = nn.Dropout(0.2)

    self.affine3 = nn.Linear(512, 128)
    self.act3 = nn.ReLU()
    self.batch3 = nn.BatchNorm1d(128)
    self.dropout3 = nn.Dropout(0.2)

    self.affine4 = nn.Linear(128, 32)
    self.act4 = nn.ReLU()
    self.batch4 = nn.BatchNorm1d(32)
    self.dropout4 = nn.Dropout(0.2)

    self.affine5 = nn.Linear(32, 10)

  def forward(self, x):
    x = x.view(-1, 32*32*3) # MLP가 이미지를 처리할 수 있도록 3차원 -> 1차원 벡터로 Flatten
    # TODO: forward pass 정의

    x = self.dropout1(self.batch1(self.act1(self.affine1(x))))
    x = self.dropout2(self.batch2(self.act2(self.affine2(x))))
    x = self.dropout3(self.batch3(self.act3(self.affine3(x))))
    x = self.dropout4(self.batch4(self.act4(self.affine4(x))))

    x = self.affine5(x)

    return x

model = MLP().to(device)

In [59]:
from tqdm.notebook import tqdm

# TODO: model 명의로 생성된 MLP 모델에 대해 학습 수행하기

lr = 8e-2
epochs = 100

optimizer = torch.optim.RMSprop(model.parameters(), lr)
loss_fn = torch.nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)

for epoch in tqdm(range(epochs), "epochs", leave=True):
    for image, label in tqdm(train_loader, "batch", leave=False):
        image, label = image.to(device), label.to(device)
        output = model(image)

        loss = loss_fn(output, label)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    scheduler.step()

    num_correct = 0

    for image, label in tqdm(val_loader, "val", leave=False):
        image, label = image.to(device), label.to(device)

        output = model(image)

        pred = output.argmax(dim=1)
        num_correct += (pred == label).sum()

    print(f"Epoch {epoch + 1} / loss: {scheduler.get_last_lr()[0]:.6e} / Acc: {num_correct / (len(train_dataset) - subset) * 100:.2f}%")

epochs:   0%|          | 0/100 [00:00<?, ?it/s]

batch:   0%|          | 0/250 [00:00<?, ?it/s]

val:   0%|          | 0/63 [00:00<?, ?it/s]

Epoch 1 / loss: 7.998026e-02 / Acc: 23.00%


batch:   0%|          | 0/250 [00:00<?, ?it/s]

...

val:   0%|          | 0/63 [00:00<?, ?it/s]

Epoch 99 / loss: 1.973759e-05 / Acc: 40.90%


batch:   0%|          | 0/250 [00:00<?, ?it/s]

val:   0%|          | 0/63 [00:00<?, ?it/s]

Epoch 100 / loss: 0.000000e+00 / Acc: 40.65%


In [60]:
# 학습된 모델 평가  *** 해당 cell을 수정하지 말 것 ***


test_dataset = datasets.CIFAR10(root='.', train=False, download=True,
                                 transform=transforms.ToTensor())
batchsize = 64
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                           batch_size=batchsize, shuffle=True)

num_correct = 0

model.eval()

with torch.no_grad():
  for image, label in test_loader:
    image, label = image.to(device), label.to(device)
    output = model(image)
    pred = output.argmax(dim=1)
    num_correct += (pred == label).sum()

print(f'Accuracy : {num_correct / len(test_dataset) * 100:.2f} %')

Files already downloaded and verified
Accuracy : 63.14 %
